# Artik 47

In [122]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import ElasticNet, Lasso, HuberRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.decomposition import PCA
from sklearn import neighbors

from tqdm import tqdm
from datetime import *
# import catboost as ctb
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1696711767661656708
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2255906407
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8743940847690836762
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [129]:
PATH_RESULTS = f'~\\Documents\\Artik_47\\Artik-47\\results\\'

## Data preparation

In [133]:
gbpusd = pd.read_csv(f'~\Documents\Artik_47\Artik-47\data\GBPUSD_M30.csv', sep='\t', parse_dates=['Time'])

#Just for close values as objetive
gbpusd = gbpusd[['Time', 'Close', 'Volume']]
gbpusd['Close'] = gbpusd['Close'].round(decimals=4)
gbpusd

,Time,Close,Volume
0,2007-01-01 06:00:00,1.9582,1728
1,2007-01-01 06:30:00,1.9582,1651
2,2007-01-01 07:00:00,1.9582,1352
3,2007-01-01 07:30:00,1.9582,1722
4,2007-01-01 08:00:00,1.9585,1729
...,...,...,...
193895,2022-07-13 03:30:00,1.1886,596
193896,2022-07-13 04:00:00,1.1881,1522
193897,2022-07-13 04:30:00,1.1879,1852
193898,2022-07-13 05:00:00,1.1880,1404


In [134]:
gbpusd.describe()

,Close,Volume
count,193900.000000,193900.000000
mean,1.513250,5233.197968
std,0.215370,11369.920694
min,1.142900,1.000000
25%,1.322375,1562.000000
50%,1.517800,2935.000000
75%,1.613200,5508.000000
max,2.114300,556867.000000


In [135]:
def shift_col(df, col_name, n):
    """
    Shift column in df, n times
    :param df: dataframe
    :param col_name: str
    :param n: int
    :return: dateframe, list columns with shift
    """
    list_shift = []
    for i in range(1,n):
        new_name_col = col_name + '_' + str(i)
        df[new_name_col] = df[col_name].shift(i)
        list_shift.append(new_name_col)

    return df, list_shift

In [136]:
df = gbpusd.copy()
df = df.rename(columns={'Time':'ds'})

#Shift last x values
last_x = 6

df, list_shift = shift_col(df, 'Close', last_x)

#Variance of shift values
df['var_last_val'] = df[list_shift].var(axis=1)

#Shift volume because is impossible to have on time
df, _ = shift_col(df, 'Volume', last_x)
df = df.drop(columns=['Volume'])

df = df.dropna()
df#.iloc[:,2:].values

,ds,Close,Close_1,Close_2,Close_3,Close_4,Close_5,var_last_val,Volume_1,Volume_2,Volume_3,Volume_4,Volume_5
5,2007-01-01 08:30:00,1.9588,1.9585,1.9582,1.9582,1.9582,1.9582,1.800000e-08,1729.0,1722.0,1352.0,1651.0,1728.0
6,2007-01-01 09:00:00,1.9588,1.9588,1.9585,1.9582,1.9582,1.9582,7.200000e-08,1524.0,1729.0,1722.0,1352.0,1651.0
7,2007-01-01 09:30:00,1.9588,1.9588,1.9588,1.9585,1.9582,1.9582,9.000000e-08,1565.0,1524.0,1729.0,1722.0,1352.0
8,2007-01-01 10:00:00,1.9584,1.9588,1.9588,1.9588,1.9585,1.9582,7.200000e-08,1704.0,1565.0,1524.0,1729.0,1722.0
9,2007-01-01 10:30:00,1.9582,1.9584,1.9588,1.9588,1.9588,1.9585,3.800000e-08,1643.0,1704.0,1565.0,1524.0,1729.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
193895,2022-07-13 03:30:00,1.1886,1.1888,1.1883,1.1888,1.1888,1.1887,4.700000e-08,448.0,796.0,979.0,2881.0,2673.0
193896,2022-07-13 04:00:00,1.1881,1.1886,1.1888,1.1883,1.1888,1.1888,4.800000e-08,596.0,448.0,796.0,979.0,2881.0
193897,2022-07-13 04:30:00,1.1879,1.1881,1.1886,1.1888,1.1883,1.1888,9.700000e-08,1522.0,596.0,448.0,796.0,979.0
193898,2022-07-13 05:00:00,1.1880,1.1879,1.1881,1.1886,1.1888,1.1883,1.330000e-07,1852.0,1522.0,596.0,448.0,796.0


In [137]:
#Find the support and resistance

#Top most frequent values with only 3 decimals
df['Close'].astype(str).str[:4].value_counts()[:10]

1.30    7039
1.56    6743
1.29    6644
1.31    6333
1.61    5880
1.60    5790
1.55    5690
1.59    5078
1.54    4994
1.32    4710
Name: Close, dtype: int64

In [77]:
#Spit
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,2:].values, df.iloc[:,1].values, test_size=0.33, shuffle=False)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train:{y_train.shape}, y_test:{y_test.shape}")

X_train: (129909, 11), X_test: (63986, 11), y_train:(129909,), y_test:(63986,)


## ML  version

In [149]:
#Fecha inicial
inicial_date = '01/01/2017 00:00:00'
inicial_date = datetime.strptime(inicial_date, '%d/%m/%Y %H:%M:%S')
#Fecha final
end_date = '01/02/2017 23:00:00'
end_date = datetime.strptime(end_date, '%d/%m/%Y %H:%M:%S')

#Nombre experimiento
NOM_EXP = 'artik47_ML'

#Variables generales de los modelos
retrain_days = [0]    #Monday is 0 and Sunday is 6
cutoff = 30         #Dias atras del index que se toman para entrenar el modelo en min

In [150]:
def backtest_regression(model_name, fecha_inicio, fecha_fin, data, cutoff, retrain_days, t_scaler, n_pca, params=None):
    # print("from: ", fecha_inicio, " to: ", fecha_fin)

    #Primer entreno
    mask = (data.ds < (fecha_inicio - timedelta(minutes=cutoff)))
    train = data.loc[mask]

    train = train.drop(columns=['ds'])

    val_reales = np.array([])
    val_pred = np.array([])
    val_fechas = np.array([])

    # Divide en variables de entrada y salida
    X, y = train.values[:, 1:], train.values[:, 0].astype('float64')

    #Scalers
    if(t_scaler=='MinMaxScaler'):
        scaler = MinMaxScaler(feature_range=(0, 1))
        X = scaler.fit_transform(X)
    elif(t_scaler=='StandardScaler'):
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    elif(t_scaler=='RobustScaler'):
        scaler = RobustScaler(quantile_range=(0, 95.0))
        X = scaler.fit_transform(X)

    #PCA
    if (type(n_pca) == int):
        pca = PCA(n_components=n_pca)
        X = pca.fit_transform(X)

    #Crea el modelo
    model = eval(model_name + "()")

    #Agrega parametros
    if(params!=None):
        model.set_params(**params)

    model.fit(X, y)

    pbar = tqdm(total=(fecha_fin-fecha_inicio).days+1)
    fecha_index = fecha_inicio
    while(fecha_index <= fecha_fin):
        # print(fecha_index)
        if(fecha_index.weekday() in retrain_days):

            #Re entreno
            mask = (data.ds < (fecha_index - timedelta(days=cutoff)))
            train = data.loc[mask]
            #print("Train: \n", train.iloc[:,0:6], "\n")
            train = train.drop(columns=['ds'])

            # Divide en variables de entrada y salida
            X, y = train.values[:, 1:], train.values[:, 0].astype('float64')

            #Scalers
            if(t_scaler=='MinMaxScaler'):
                scaler = MinMaxScaler(feature_range=(0, 1))
                X = scaler.fit_transform(X)
            elif(t_scaler=='StandardScaler'):
                scaler = StandardScaler()
                X = scaler.fit_transform(X)
            elif(t_scaler=='RobustScaler'):
                scaler = RobustScaler(quantile_range=(0, 95.0))
                X = scaler.fit_transform(X)

            #PCA
            if (type(n_pca) == int):
                pca = PCA(n_components=n_pca)
                X = pca.fit_transform(X)

            model = eval(model_name + "()")

            if(params!=None):
                model.set_params(**params)

            model.fit(X, y)

        mask = (data.ds >= fecha_index) & (data.ds <= fecha_index + timedelta(hours=23))
        to_pred = data.loc[mask]
        if not(to_pred.empty):

            val_fechas = np.append(val_fechas, to_pred.iloc[:,0].tolist())
            #print("to_pred: ", to_pred.values)
            to_pred = to_pred.drop(columns=['ds'])
            # Divide en variables de entrada y salida
            X, y = to_pred.values[:, 1:], to_pred.values[:, 0].astype('float64')

            #Scalers
            if((t_scaler=='MinMaxScaler') or (t_scaler=='StandardScaler') or (t_scaler=='RobustScaler')):
                X = scaler.transform(X)

            #PCA
            if (type(n_pca) == int):
                X = pca.transform(X)

            y_hat = model.predict(X)

            val_reales = np.append(val_reales, y)
            val_pred = np.append(val_pred, y_hat)

        fecha_index = fecha_index + timedelta(days=1)
        pbar.update(1)

    resultados = pd.DataFrame({'fechas':val_fechas, 'val_reales':val_reales, 'val_pred':val_pred})
    pbar.close()
    return resultados

In [ ]:
models_dict = {
    # 'XGBoost0':{'xgb.XGBRegressor':{'n_jobs': 10}, 'Scaler':False, 'PCA':False},
    # 'XGBoost1':{'xgb.XGBRegressor':{'Objective':'reg:squaredlogerror',
    #                                'n_estimator':100,
    #                                # 'eval_metric':'mape',
    #                                'booster':'gbtree',
    #                                'verbosity':0,
    #                                # 'disable_default_eval_metric':'false',
    #                                'learning_rate':0.2,
    #                                'max_depth':7,
    #                                'min_child_weight':2,
    #                                'sampling_method':'gradient_based',
    #                                'n_jobs': 10}, 'Scaler':False, 'PCA':False},
    #'ElasticNet':{'ElasticNet':{'alpha':1.0, 'l1_ratio':0.5}},
    #'ElasticNet_2':{'ElasticNet':None},
    #'Lasso':{'Lasso':None},
    'RandomForest':{'RandomForestRegressor':{'n_jobs': 4}, 'Scaler':False, 'PCA':False},
    # 'RandomForest_2':{'RandomForestRegressor':{'n_estimators': 300, 'min_samples_split': 10,
    #                                         'min_samples_leaf': 1, 'max_features': 'sqrt','max_depth': 40,
    #                                       'bootstrap': False, 'n_jobs': 10}, 'Scaler':False, 'PCA':False},
    #                'AdaBoost':{'AdaBoostRegressor':None},
    # 'SVR0':{'SVR':None, 'Scaler':'StandardScaler', 'PCA':5},
    # 'SVR1':{'SVR':None, 'Scaler':'StandardScaler', 'PCA':9},
    # 'SVR2':{'SVR':None, 'Scaler':'StandardScaler', 'PCA':10},
    # 'HuberRegressor':{'HuberRegressor':{'epsilon':1.0, 'max_iter':200*20, 'alpha':0.0001}, 'Scaler':False, 'PCA':False},
    # 'HuberRegressor1':{'HuberRegressor':None, 'Scaler':'StandardScaler', 'PCA':9},
    # 'HuberRegressor2':{'HuberRegressor':None, 'Scaler':False, 'PCA':9},
    # 'HuberRegressor2':{'HuberRegressor':{'epsilon':1.0, 'max_iter':200*20, 'alpha':1e-10}, 'Scaler':False, 'PCA':9},
    # 'HuberRegressor3':{'HuberRegressor':None, 'Scaler':'StandardScaler', 'PCA':9},
    # 'HuberRegresso4':{'HuberRegressor':{'epsilon':1.0, 'max_iter':200*20, 'alpha':1e-10}, 'Scaler':'StandardScaler', 'PCA':9},
    # 'HuberRegressor5':{'HuberRegressor':None, 'Scaler':'StandardScaler', 'PCA':9},
    # 'SVR2':{'SVR':None, 'Scaler':'RobustScaler', 'PCA':6},
    #'KNR':{'neighbors.KNeighborsRegressor':None},
    #'BaggingRegressor':{'BaggingRegressor':None},
    #'GradientBoosting':{'GradientBoostingRegressor':None},
}

df_resultados = pd.DataFrame()

for modelo in models_dict.items():
    #print(modelo, type(modelo))
    try:
        output_name_model = modelo[0]
        name_model = list(modelo[1])[0]
        params = modelo[1].get(name_model)
        scaler = modelo[1]['Scaler']
        pca = modelo[1]['PCA']

        start_time = datetime.now()
        print("\n\n--->", output_name_model, '<--- hora inicio:', start_time)
        print("Parametros: ", params, " scaler: ", scaler, " pca: ", pca)
        nombre = output_name_model + '_' + NOM_EXP + '_' + start_time.strftime('%Y_%m_%d-%H_%M') + '.csv'
        print(nombre)
        resultados = backtest_regression(name_model, inicial_date, end_date, df, cutoff, retrain_days,
                                         scaler, pca, params)
        resultados = resultados.round(decimals=4)
        print(resultados)
        print("mean absolute porcentage error:", mean_absolute_percentage_error(resultados['val_reales'], resultados['val_pred'])*100)
        print("mean absolute error:", mean_absolute_error(resultados['val_reales'], resultados['val_pred']))
        # print("root mean square error:", root_mean_square_error(resultados))
        print("tiempo de ejecucion: ", datetime.now() - start_time)
        resultados.to_csv(PATH_RESULTS + nombre, index = False, header=True)

    except Exception as ex:
        print(":::::::::::  Error en " + modelo[0], " ::: ", ex)



---> RandomForest <--- hora inicio: 2022-10-01 22:54:10.308712
Parametros:  {'n_jobs': 4}  scaler:  False  pca:  False
RandomForest_artik47_ML_2022_10_01-22_54.csv


 19%|█▉        | 6/32 [00:32<01:50,  4.26s/it]

In [141]:
df.loc[(df['ds'] >= '2008-01-01 00:00:00') & (df['ds'] <= '2008-01-02 23:00:00')]

,ds,Close,Close_1,Close_2,Close_3,Close_4,Close_5,var_last_val,Volume_1,Volume_2,Volume_3,Volume_4,Volume_5
12516,2008-01-01 00:00:00,1.9821,1.9824,1.9826,1.9840,1.9872,1.9908,1.280000e-05,3833.0,3714.0,9333.0,14286.0,18370.0
12517,2008-01-01 00:30:00,1.9837,1.9821,1.9824,1.9826,1.9840,1.9872,4.448000e-06,2204.0,3833.0,3714.0,9333.0,14286.0
12518,2008-01-01 01:00:00,1.9839,1.9837,1.9821,1.9824,1.9826,1.9840,7.030000e-07,1567.0,2204.0,3833.0,3714.0,9333.0
12519,2008-01-01 01:30:00,1.9847,1.9839,1.9837,1.9821,1.9824,1.9826,6.530000e-07,1336.0,1567.0,2204.0,3833.0,3714.0
12520,2008-01-01 02:00:00,1.9848,1.9847,1.9839,1.9837,1.9821,1.9824,1.178000e-06,1165.0,1336.0,1567.0,2204.0,3833.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12606,2008-01-02 21:00:00,1.9806,1.9784,1.9786,1.9798,1.9807,1.9818,2.048000e-06,16191.0,18928.0,15231.0,18075.0,11442.0
12607,2008-01-02 21:30:00,1.9791,1.9806,1.9784,1.9786,1.9798,1.9807,1.172000e-06,19885.0,16191.0,18928.0,15231.0,18075.0
12608,2008-01-02 22:00:00,1.9806,1.9791,1.9806,1.9784,1.9786,1.9798,8.200000e-07,12797.0,19885.0,16191.0,18928.0,15231.0
12609,2008-01-02 22:30:00,1.9821,1.9806,1.9791,1.9806,1.9784,1.9786,1.148000e-06,21229.0,12797.0,19885.0,16191.0,18928.0
